In [1]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [19]:
from torch_geometric.nn import GCNConv


In [31]:
import torch
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        print(x.size(), edge_index.size())
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        print(x)
        print(x.size())
        return F.log_softmax(x, dim=1)

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

data = dataset[0].to(device)
print(data)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    break

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
torch.Size([2708, 1433]) torch.Size([2, 10556])
tensor([[ 0.0060,  0.0212, -0.0099,  ..., -0.0450, -0.0178,  0.0394],
        [-0.0587,  0.0201, -0.0486,  ...,  0.0041, -0.0612,  0.0895],
        [-0.1164, -0.0191, -0.1055,  ...,  0.0624, -0.0504,  0.0543],
        ...,
        [-0.1947,  0.0467, -0.2435,  ..., -0.0645,  0.0677, -0.0006],
        [-0.0961,  0.0004, -0.0905,  ...,  0.0448,  0.0114, -0.0146],
        [-0.0514, -0.0053, -0.0608,  ..., -0.0020,  0.0098, -0.0216]],
       device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([2708, 7])


In [26]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

torch.Size([2708, 1433]) torch.Size([2, 10556])
Accuracy: 0.3920
